In [ ]:
#this script is used to create the table trip_table_fixed used in other scripts
#trip_table_fixed table will have added columnes to be used for creating DFS

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time
from datetime import datetime, timedelta



In [2]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
weekday_stop_time_fixed =  pd.read_csv(r'weekday_stop_time_fixed.csv', sep=",", engine='python')
weekday_stop_time_fixed.head()

,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled,arrival_time_:,departure_time_:,time_diff,seq,total_time,arrival_time_fix,departure_time_fix
0,884475,1,2561,0,1,9:57:00,9:57:00,1,CAPITOL SQUARE,0.0210,95700,95700,NaN,NaN,0,35820,35820
1,884475,2,2605,0,0,9:57:55,9:57:55,0,CAPITOL SQUARE,0.1979,95755,95755,NaN,NaN,0,35875,35875
2,884475,3,2717,0,0,9:58:51,9:58:51,0,CAPITOL SQUARE,0.3775,95851,95851,NaN,NaN,0,35931,35931
3,884475,4,2921,0,0,9:59:51,9:59:51,0,CAPITOL SQUARE,0.5692,95951,95951,NaN,NaN,0,35991,35991
4,884475,5,133,0,0,10:01:00,10:01:00,1,CAPITOL SQUARE,0.7875,100100,100100,NaN,NaN,0,36060,36060


In [3]:
# route_short_name == the bus number
# the trip_id can be linked with the trip id in stop times
# trip id is unique to each row. 
trips =  pd.read_csv(r'C:\\Users\\이진우\\cs_summer\\Jin_project\\mmt_gtfs\\trips.txt', sep=",", engine='python')
trips.head()

,route_id,route_short_name,service_id,trip_id,trip_headsign,direction_id,direction_name,block_id,shape_id,shape_code,trip_type,trip_sort,wheelchair_accessible,bikes_allowed
0,8380,1,86_WKD,884475,CAPITOL SQUARE,0,Capitol Square,183766,51362,1E,NaN,35820,1,1
1,8380,1,86_WKD,884474,CAPITOL SQUARE,0,Capitol Square,183765,51362,1E,NaN,39420,1,1
2,8380,1,86_WKD,884473,CAPITOL SQUARE,0,Capitol Square,183766,51362,1E,NaN,43020,1,1
3,8380,1,86_WKD,884472,CAPITOL SQUARE,0,Capitol Square,183765,51362,1E,NaN,46620,1,1
4,8380,1,86_WKD,884471,CAPITOL SQUARE,0,Capitol Square,183766,51362,1E,NaN,50220,1,1


In [4]:
list_of_stops = weekday_stop_time_fixed['stop_id'].unique()
# weekday_stop_time_fixed = weekday_stop_time_fixed[weekday_stop_time_fixed['arrival_time_fix'].between(25200, 32400, inclusive=True)]
list_of_row = []
weekday_stop_time_fixed['route_short_name'] = 0
weekday_stop_time_fixed['seq'] = 0
weekday_stop_time_fixed['time_diff'] = 0
unique_id=trips.groupby(['trip_id','route_short_name']).size().reset_index()

In [5]:
for index, row in unique_id.iterrows():
    trip_id=row['trip_id']
    bus_number = row['route_short_name']
    weekday_stop_time_fixed.loc[weekday_stop_time_fixed['trip_id']==trip_id, 'route_short_name'] = bus_number

In [11]:
trip_table=copy.deepcopy(weekday_stop_time_fixed)
trip_table.head()

,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled,arrival_time_:,departure_time_:,time_diff,seq,total_time,arrival_time_fix,departure_time_fix,route_short_name
0,884475,1,2561,0,1,9:57:00,9:57:00,1,CAPITOL SQUARE,0.0210,95700,95700,0,0,0,35820,35820,1
1,884475,2,2605,0,0,9:57:55,9:57:55,0,CAPITOL SQUARE,0.1979,95755,95755,0,0,0,35875,35875,1
2,884475,3,2717,0,0,9:58:51,9:58:51,0,CAPITOL SQUARE,0.3775,95851,95851,0,0,0,35931,35931,1
3,884475,4,2921,0,0,9:59:51,9:59:51,0,CAPITOL SQUARE,0.5692,95951,95951,0,0,0,35991,35991,1
4,884475,5,133,0,0,10:01:00,10:01:00,1,CAPITOL SQUARE,0.7875,100100,100100,0,0,0,36060,36060,1


In [12]:
unique_head=trip_table.groupby(['stop_headsign','route_short_name']).size().reset_index()
for index, row in unique_head.iterrows():
    stop_headsign = unique_head.loc[index, 'stop_headsign']
    bus_number = unique_head.loc[index,'route_short_name']
    full_bus_name = str(bus_number) + '_' + str(stop_headsign)
    trip_table.loc[((trip_table['stop_headsign'] == stop_headsign) & (trip_table['route_short_name'] == bus_number)), 'full_bus_name'] = full_bus_name 


In [13]:
#this section takes a long time
for index, row in trip_table.iterrows():
    arrival_time= trip_table.loc[index, 'arrival_time_fix']
    stop_id = trip_table.loc[index,'stop_id']
    stop_id_time = str(stop_id) + '_' + str(arrival_time)
    trip_table.loc[index, 'stop_id_time'] = stop_id_time 


In [14]:
trip_table_fixed = copy.deepcopy(trip_table)
trip_table_fixed.to_csv('trip_table_fixed.csv', encoding='utf-8', index=False)